In [25]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [2]:
tf.__version__

'2.12.0'

In [ ]:
#PART 1 - DATA PREPROCESSING

In [3]:
#APPLY TRANSFORMATIONS ON TRAINING SET TO AVOID OVERFITTING

#-> GEOMETRICAL TRANSFORMATIONS; ZOOM IN, OUT; AUGMENT IMAGES
#-> IMAGE AUGMENTATION [ON TRAINING SET]

train_datagen = ImageDataGenerator(
    #COMPULSARY FEATURE SCALING (0-1)
                    rescale = 1./255, #FEATURE SCALING; DIVIDE EACH PIXEL VALUE BY 255
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True
)


training_set = train_datagen.flow_from_directory(
                'dataset/training_set', #PATH TO FOLDER CONTAINING ALL IMAGES
                target_size = (64,64),    #FINAL SIZE OF IMAGE WHEN FED TO CNN [HIGHER INCREASES COMPUTATIONAL TIME]
                batch_size = 32,
                class_mode = 'binary' #ONLY 2 OUTCOMES; ELSE 'categorical'
)
    
                

# ONLY SCALING ON THE TESTING SET [NO AUGMENTATION]
test_datagen = ImageDataGenerator(
    rescale = 1./255
)

test_set = test_datagen.flow_from_directory(
            'dataset/test_set',
            target_size = (64,64),
            batch_size = 32,
            class_mode = 'binary'
)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [6]:
# PART 2 - BUILDING THE CNN


#INITIALISING THE CNN
cnn = Sequential()

In [9]:
#STEP 1 - CONVOLUTION
cnn.add(Conv2D(
    activation='relu', #RECTIFIER ACTIVATION FUNCTION
    input_shape = (64,64, 3), #3 BECAUSE COLOURED (RGB) #ONLY IN 1ST LAYER
    filters  = 32, #NUMBER OF FEATURES
    kernel_size = 3 # 3X3 ARRAY
))

In [11]:
#STEP 2 - POOLING
cnn.add(
    MaxPool2D(
    pool_size = 2, #2X2 ARRAY
    strides=2 #SHIFT BY 2 PIXEL EACH TIME
    )
)

In [12]:
# SECOND CONVOLUTIONAL LAYER
cnn.add(Conv2D(
    activation='relu', #RECTIFIER ACTIVATION FUNCTION
    filters  = 32, #NUMBER OF FEATURES
    kernel_size = 3 # 3X3 ARRAY
))

In [14]:
# STEP - 3 - FLATTENING
cnn.add(
    Flatten()
)

In [16]:
# STEP - 4 - FULLY CONNECTED LAYER
cnn.add(
    Dense(
        units = 128,
        activation = 'relu'
    )
)

In [18]:
# STEP - 5 - OUTPUT LAYER [FULLY CONNECTED TO HIDDEN LAYER]
cnn.add(
    Dense(
        units = 1, #BINARY CLASSIFICATION
        activation = 'sigmoid' #FOR MULTI-CLASS: SOFTMAX
    )
    
)

In [19]:
# PART - 3: TRAINING THE CNN


#COMPILING THE CNN MODEL
cnn.compile(
    optimizer = 'adam', #SGD OPTIMIZER
    loss = 'binary_crossentropy', #BINARY CLASSIFICATION
    metrics = ['accuracy']
)

In [21]:
# TRAINING THE CNN MODEL ON TRAINING_SET
cnn.fit(
    x=training_set,
    validation_data=test_set,
    epochs = 25
)

Epoch 1/25
  1/250 [..............................] - ETA: 44s - loss: 0.4780 - accuracy: 0.7812

2023-06-24 23:18:12.157699: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


250/250 [==============================] - 49s 197ms/step - loss: 0.3854 - accuracy: 0.8236 - val_loss: 0.5028 - val_accuracy: 0.7790
Epoch 2/25
250/250 [==============================] - 52s 206ms/step - loss: 0.3679 - accuracy: 0.8351 - val_loss: 0.5217 - val_accuracy: 0.7730
Epoch 3/25
250/250 [==============================] - 45s 182ms/step - loss: 0.3506 - accuracy: 0.8401 - val_loss: 0.5220 - val_accuracy: 0.7720
Epoch 4/25
250/250 [==============================] - 44s 175ms/step - loss: 0.3480 - accuracy: 0.8428 - val_loss: 0.5740 - val_accuracy: 0.7345
Epoch 5/25
250/250 [==============================] - 40s 162ms/step - loss: 0.3384 - accuracy: 0.8511 - val_loss: 0.5178 - val_accuracy: 0.7825
Epoch 6/25
250/250 [==============================] - 40s 161ms/step - loss: 0.3352 - accuracy: 0.8549 - val_loss: 0.5233 - val_accuracy: 0.7885
Epoch 7/25
250/250 [==============================] - 41s 164ms/step - loss: 0.3094 - accuracy: 0.8636 - val_loss: 0.5232 - val_accuracy: 0.7

In [43]:
# MAKING A SINGLE PREDICTION

#LOAD INPUT IMAGE: PIL FORMAT: INPUT SHAPE IS IMPORTANT
test_image = image.load_img(
        "dataset/single_prediction/download-4.jpg", 
        target_size = (64, 64)
    ) 

#PIL TO NUMPY ARRAY
test_image = image.img_to_array(
    test_image
)

#CONVERT IMAGE TO A BATCH (AS BATCH_SIZE > 1 IN TRAINING & TESTING SET)
test_image = np.expand_dims(
    test_image,
    axis = 0 #DIMENSION OF BATCH WILL BE THE FIRST DIMENSION
)

#PREDICTION [PROBABILITY]
result = cnn.predict(
    test_image/255.0
)

#CLASS CORRESPONDING TO 0 & TO 1
training_set.class_indices


if result[0][0] > 0.5:   # 0-> BATCH 0 -> 1ST & ONLY PREDICTION
    prediction = 'dog'
    
else:
    prediction = 'cat'
    
print(prediction)

1/1 [==============================] - 0s 29ms/step
cat
